In [7]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [9]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
print(model)

# print parameters
module_parameters = filter(lambda p: p.requires_grad, model.parameters())
param_cnt = sum([np.prod(p.size()) for p in module_parameters])
print('total trainable params: %d' % param_cnt)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
total trainable params: 397510


In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3755
Epoch [1/5], Step [200/600], Loss: 0.3277
Epoch [1/5], Step [300/600], Loss: 0.1696
Epoch [1/5], Step [400/600], Loss: 0.1323
Epoch [1/5], Step [500/600], Loss: 0.1921
Epoch [1/5], Step [600/600], Loss: 0.2093
Epoch [2/5], Step [100/600], Loss: 0.1256
Epoch [2/5], Step [200/600], Loss: 0.1120
Epoch [2/5], Step [300/600], Loss: 0.1276
Epoch [2/5], Step [400/600], Loss: 0.1184
Epoch [2/5], Step [500/600], Loss: 0.0575
Epoch [2/5], Step [600/600], Loss: 0.0531
Epoch [3/5], Step [100/600], Loss: 0.1071
Epoch [3/5], Step [200/600], Loss: 0.0415
Epoch [3/5], Step [300/600], Loss: 0.0301
Epoch [3/5], Step [400/600], Loss: 0.0843
Epoch [3/5], Step [500/600], Loss: 0.0414
Epoch [3/5], Step [600/600], Loss: 0.0347
Epoch [4/5], Step [100/600], Loss: 0.0539
Epoch [4/5], Step [200/600], Loss: 0.0113
Epoch [4/5], Step [300/600], Loss: 0.0684
Epoch [4/5], Step [400/600], Loss: 0.0320
Epoch [4/5], Step [500/600], Loss: 0.1281
Epoch [4/5], Step [600/600], Loss:

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')